In [26]:
# Importing necessary libraries and settings
import time
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [27]:
# Insert your Spotify username and the credentials that you obtained from spotify developer
cid = 'XXXXXX'
secret = 'XXXXXXX'
redirect_uri='http://localhost:7777/callback'
username = 'XXXXXXX'

In [28]:
# Once the Authorisation is complete, we just need to `sp` to call the APIs
scope = 'user-top-read playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [29]:
# Getting features for each song
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

### Getting the songs from all of Spotify's playlists

The following cells are collecting the songs frm all of the playlists from the official Spotify account. In this repo, I've curated a dataframe of approximately 10,000 songs. If you would like to add more songs, modify the cells below. I've commented them out because they take a long time to run and are unnecessary if you'd like to use the dataframe I've already built.

In [30]:
# # Getting playlist IDs from each of Spotify's playlists
# playlists = sp.user_playlists('spotify')
# spotify_playlist_ids = []
# while playlists:
#     for i, playlist in enumerate(playlists['items']):
#         spotify_playlist_ids.append(playlist['uri'][-22:])
#     if playlists['next']:
#         playlists = sp.next(playlists)
#     else:
#         playlists = None
# spotify_playlist_ids[:20]

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [31]:
# len(spotify_playlist_ids)

1398

### Getting tracks from Spotify playlists

In [32]:
# Creating a function to get the first 50 tracks IDs from a playlist
def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item['track']
        ids.append(track['id'])
    return

In [33]:
# Creating a function get features of each track from track id
def getTrackFeatures(track_id):
  meta = sp.track(track_id)
  features = sp.audio_features(track_id)

  # meta
  track_id = track_id
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [track_id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

The cell below takes about five minutes to run.

In [34]:
# %%time
# # Gathering track ids
# ids = []
# for x in spotify_playlist_ids[:200]:
#     getTrackIDs(x)
# ids[:5]

The cell below takes about 30 minutes to run.

In [35]:
# %%time
# # loop over track ids to get audio features for each track
# tracks = []
# for i in range(len(ids)):
#     try:  
#         track = getTrackFeatures(ids[i])
#         tracks.append(track)
#     except:
#         pass

# # create dataset
# df = pd.DataFrame(tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
# df.head()

In [36]:
# df.to_csv('playlist_songs.csv',index=False)

In [37]:
df = pd.read_csv('data/playlist_songs.csv')
df.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,Leave The Door Open,Leave The Door Open,Bruno Mars,2021-03-05,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,Save Your Tears,After Hours,The Weeknd,2020-03-20,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,We're Good,Future Nostalgia (The Moonlight Edition),Dua Lipa,2021-02-11,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,Hold On,Hold On,Justin Bieber,2021-03-05,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,Astronaut In The Ocean,Masked Wolf,2021-01-06,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [38]:
# Dropping columns that could lead to data leakage
df = df.drop(columns=['name', 'album', 'artist', 'release_date'])
df.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7MAibcTli4IisCtbHKrGMh,242096,90,0.586,0.1820,0.616,0.000000,0.0927,-7.964,0.0324,148.088,4
1,5QO79kh1waicV47BqGRL3g,215626,97,0.680,0.0212,0.826,0.000012,0.5430,-5.487,0.0309,118.051,4
2,1diS6nkxMQc3wwC4G1j0bh,165506,88,0.722,0.0319,0.588,0.000000,0.1830,-5.932,0.0544,134.010,4
3,4u4NyuceXP7Uzh7XFJKCr1,170813,89,0.658,0.0106,0.634,0.000000,0.1320,-5.797,0.0413,139.980,4
4,3Ofmpyhv5UAQ70mENzB277,132780,94,0.778,0.1750,0.695,0.000000,0.1500,-6.865,0.0913,149.996,4


In [39]:
df.shape

(9769, 12)

## Getting user's favorite tracks

In [40]:
# Getting top 50 tracks from user
results = sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term')

In [41]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head()

,track_name,album,track_id,artist,duration,popularity
0,Place,Whole Lotta Red,1Bg2CNZw6S4e9cGWPmi0uI,Playboi Carti,117239,69
1,Feels Like Death,Feels Like Death,03vMyCyCK7pVWjC1i1zur0,Levi Carter,237505,45
2,Glory Boy,Freewave 3,5LpnrXjrt0BOU0iOGH78UN,LUCKI,111048,41
3,Tarantino,Almost There,1CzjJzPOCLkZr2oqiOrXjc,LUCKI,114773,38
4,Faith,Faith,0TqNfrOY2IrpFRI2zxsMq4,LUCKI,107467,37


In [42]:
# Getting track features for each song in favorite song dataframe
%%time
fav_tracks = []
for track in df_favourite['track_id']:
    try:  
        track = getTrackFeatures(track)
        fav_tracks.append(track)
    except:
        pass

CPU times: user 405 ms, sys: 64.1 ms, total: 469 ms
Wall time: 6.56 s


In [43]:
# Create favorite track wtih audio features dataset
df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_fav.head()

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,1Bg2CNZw6S4e9cGWPmi0uI,Place,Whole Lotta Red,Playboi Carti,2020-12-25,117239,69,0.891,0.000674,0.628,0.0000,0.1190,-6.350,0.105,131.060,4
1,03vMyCyCK7pVWjC1i1zur0,Feels Like Death,Feels Like Death,Levi Carter,2018-04-24,237505,45,0.755,0.337000,0.626,0.0000,0.1370,-6.247,0.293,110.088,4
2,5LpnrXjrt0BOU0iOGH78UN,Glory Boy,Freewave 3,LUCKI,2019-02-15,111048,41,0.784,0.096600,0.519,0.0109,0.0748,-9.868,0.096,129.953,3
3,1CzjJzPOCLkZr2oqiOrXjc,Tarantino,Almost There,LUCKI,2020-05-29,114773,38,0.790,0.049500,0.547,0.0000,0.1140,-9.974,0.108,135.037,4
4,0TqNfrOY2IrpFRI2zxsMq4,Faith,Faith,LUCKI,2020-04-17,107467,37,0.878,0.123000,0.584,0.0000,0.1050,-10.014,0.419,125.069,4


In [44]:
# Dropping columns that could lead to data leakage
df_fav = df_fav.drop(columns=['name', 'album', 'artist', 'release_date'])
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,1Bg2CNZw6S4e9cGWPmi0uI,117239,69,0.891,0.000674,0.628,0.0000,0.1190,-6.350,0.105,131.060,4
1,03vMyCyCK7pVWjC1i1zur0,237505,45,0.755,0.337000,0.626,0.0000,0.1370,-6.247,0.293,110.088,4
2,5LpnrXjrt0BOU0iOGH78UN,111048,41,0.784,0.096600,0.519,0.0109,0.0748,-9.868,0.096,129.953,3
3,1CzjJzPOCLkZr2oqiOrXjc,114773,38,0.790,0.049500,0.547,0.0000,0.1140,-9.974,0.108,135.037,4
4,0TqNfrOY2IrpFRI2zxsMq4,107467,37,0.878,0.123000,0.584,0.0000,0.1050,-10.014,0.419,125.069,4


In [1]:
# Creating favorite column to use in classification
df_fav['favorite'] = 1
df['favorite'] = 0 

NameError: name 'df_fav' is not defined

In [46]:
# Checking if both datasets have the same columns
df.columns == df_fav.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

## Preparing dataset for model creation

In [47]:
print(df.shape)
print(df_fav.shape)

(9769, 13)
(50, 13)


In [48]:
# Combining the favorite dataframe with the dataframe of Spotify's songs
combined = pd.concat([df, df_fav])
combined.shape

(9819, 13)

In [49]:
combined.favorite.value_counts()

0    9769
1      50
Name: favorite, dtype: int64

The favorite songs to not favorite songs ratio is imbalanced, I will need to fix this when building the model

In [55]:
# Creating dataframe of favorite songs
df_fav = combined.loc[combined['favorite'] == 1]
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,favorite
0,1Bg2CNZw6S4e9cGWPmi0uI,117239,69,0.891,0.000674,0.628,0.0000,0.1190,-6.350,0.105,131.060,4,1
1,03vMyCyCK7pVWjC1i1zur0,237505,45,0.755,0.337000,0.626,0.0000,0.1370,-6.247,0.293,110.088,4,1
2,5LpnrXjrt0BOU0iOGH78UN,111048,41,0.784,0.096600,0.519,0.0109,0.0748,-9.868,0.096,129.953,3,1
3,1CzjJzPOCLkZr2oqiOrXjc,114773,38,0.790,0.049500,0.547,0.0000,0.1140,-9.974,0.108,135.037,4,1
4,0TqNfrOY2IrpFRI2zxsMq4,107467,37,0.878,0.123000,0.584,0.0000,0.1050,-10.014,0.419,125.069,4,1


In [56]:
# Removing favorite songs from playlist songs
df = combined.loc[combined['favorite'] != 1]
df.shape

(9769, 13)

In [57]:
df_fav.shape

(50, 13)

In [58]:
# Saving these dataframes to use in model creation
df.to_csv('encoded_playlist_songs.csv', index=False)
df_fav.to_csv('favorite_songs.csv', index=False)